In [9]:
# # Instalar las librerías necesarias para ejecutar el script
# %pip install openai
# %pip install llama-index
# %pip install pypdf
# %pip install langchain
# %pip install "unstructured[all-docs]"
# %pip install faiss-gpu
# %pip install faiss-cpu

In [10]:
pip install google.cloud.storage

Note: you may need to restart the kernel to use updated packages.


In [11]:
import openai
import os

#os.environ["OPENAI_API_KEY"] =
openai.api_key = "xxx"

In [12]:
from langchain.document_loaders import UnstructuredFileLoader
from google.cloud import storage
from google.oauth2 import service_account
import tempfile
import os

def extract_file_content(file_path):
    loader = UnstructuredFileLoader(file_path)
    documents = loader.load()
    documents_content = '\n'.join(doc.page_content for doc in documents)
    return documents_content

def download_file_from_gcs(bucket_name, file_name, service_account_json):
    """Downloads a file from Google Cloud Storage and returns its content as bytes.

    Args:
        bucket_name (str): The name of the Google Cloud Storage bucket.
        file_name (str): The name of the file to download.
        service_account_json (str): The path to the service account JSON file.

    Returns:
        str: The local path to the downloaded file.
    """
    # Load the service account credentials from the JSON file.
    credentials = service_account.Credentials.from_service_account_file(service_account_json)

    # Create a storage client with the loaded credentials.
    client = storage.Client(credentials=credentials)

    bucket = client.bucket(bucket_name)
    file_object = bucket.blob(file_name)

    # Check if the object exists and fetch its content.
    if not file_object.exists():
        raise FileNotFoundError(f"The object '{file_name}' does not exist in the bucket '{bucket_name}'.")

    # Create a temporary directory to store the downloaded file.
    temp_dir = tempfile.mkdtemp()
    local_file_path = os.path.join(temp_dir, file_name)

    # Download the file to the temporary directory.
    file_object.download_to_filename(local_file_path)

    return local_file_path

if __name__ == "__main__":
    bucket_name = "clausulados"
    file_name = "Prot_tarj.pdf"
    service_account_json = "service_account_key.json"  # Replace with the actual path to your service account JSON file.
    max_characters = 400

    # Download the file from Google Cloud Storage
    local_file_path = download_file_from_gcs(bucket_name, file_name, service_account_json)

    # Extract the content using the UnstructuredFileLoader and limit to the first 400 characters
    extracted_content = extract_file_content(local_file_path)

    # Now 'extracted_content' contains the first 400 characters of the text extracted from the PDF.
    print(f"First {max_characters} Characters of the Document: \n{extracted_content[:max_characters]}...")


First 400 Characters of the Document: 
SEGURO DE PROTECCIÓN PATRIMONIAL TARJETAHABIENTES 27/09/2021-1327-P-06-RC-0000000000046-0R00

CONDICIONES GENERALES

ASEGURADO Es el titular de la(s) tarjetas(s) débito o crédito expedidos por el B a n c o D a v i v i e n d a y q u i e n voluntariamente se adhiere a la póliza.

le Seguros Comerciales Bolívar en adelante reconocerá al o beneficiario, el pago del valor asegurado, hasta el número y v...


In [13]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator = '\n\n',
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function = len
)

In [14]:
document_chunks = text_splitter.split_text(extracted_content)
print(f"# Chunks in document :", {len(document_chunks)})

# Chunks in document : {15}


In [26]:
from vertexai.language_models import TextEmbeddingModel
from google.cloud import aiplatform

aiplatform.init(project="llama-test-396816")
def text_embedding(chunks: list) -> list:
    """Text embedding with a Large Language Model."""
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
    embeddings = []
    for chunk in chunks:
        embeddings.append(model.get_embeddings([chunk]))
    return embeddings


document_chunks = text_splitter.split_text(extracted_content)
print(f"# Chunks in document :", {len(document_chunks)})
embeddings = text_embedding(document_chunks)

print(embeddings)

# Chunks in document : {15}
